In [1]:
import numpy as np
import plotly.graph_objects as go
import uuid

In [2]:

def plot_error_symmetry(error_func):
    """
    Parameters:
    - error_func: function to calculate error given true and predicted values (e.g., mse)
    """
    # Create a range of actual values
    actual_values = np.arange(100, 0, -10)  # Actual values from 100 to 10 in decrements of 10
    predicted_values = actual_values[::-1]  # Reverse to create predicted values

    # Calculate differences centered around 0
    differences = predicted_values - actual_values
    over_diff = np.where(differences > 0, differences, np.nan)  # Positive differences (over-prediction)
    under_diff = np.where(differences < 0, differences, np.nan)  # Negative differences (under-prediction)

    # Calculate errors for each case and convert to integers
    over_error = [
        int(np.round(error_func(np.array([t]), np.array([p])))) if not np.isnan(d) else np.nan 
        for t, p, d in zip(actual_values, predicted_values, over_diff)
    ]
    under_error = [
        int(np.round(error_func(np.array([t]), np.array([p])))) if not np.isnan(d) else np.nan 
        for t, p, d in zip(actual_values, predicted_values, under_diff)
    ]

    # Create a Plotly figure
    fig = go.Figure()

    # Add trace for over-prediction errors with lines connecting the dots
    fig.add_trace(go.Scatter(
        x=over_diff,
        y=over_error,
        mode='lines+markers',  # Connect markers with lines
        marker=dict(size=10, color='rgba(255, 165, 0, 0.8)', line=dict(width=1, color='rgba(255, 165, 0, 1)')),
        name='Over-Prediction Error',
        text=[f'Pred: {pred}, Actual: {actual}, Error: {error}' 
              for pred, actual, error in zip(predicted_values, actual_values, over_error)],
        hoverinfo='text'  # Show text on hover
    ))

    # Add trace for under-prediction errors with lines connecting the dots
    fig.add_trace(go.Scatter(
        x=under_diff,
        y=under_error,
        mode='lines+markers',  # Connect markers with lines
        marker=dict(size=10, color='rgba(0, 128, 255, 0.8)', line=dict(width=1, color='rgba(0, 128, 255, 1)')),
        name='Under-Prediction Error',
        text=[f'Pred: {pred}, Actual: {actual}, Error: {error}' 
              for pred, actual, error in zip(predicted_values, actual_values, under_error)],
        hoverinfo='text'  # Show text on hover
    ))

    # Update layout for a professional appearance
    fig.update_layout(
        title=f"Error Analysis: {' '.join(error_func.__name__.split('_')).title()} for Over- and Under-Prediction",
        title_x=0.5,  # Center title
        title_font=dict(size=16, family='"Segoe UI", "Roboto", "Oxygen", "Ubuntu", "Droid Sans", "Helvetica Neue", sans-serif', weight='bold'),  # Bold title
        xaxis_title="Difference between Predicted and Actual Values",
        yaxis_title="Error",
        template="plotly_white",
        font=dict(size=14, family='"Segoe UI", "Roboto", "Oxygen", "Ubuntu", "Droid Sans", "Helvetica Neue", sans-serif'),
        showlegend=False,  # Hide the legend
        xaxis=dict(tickmode='array', gridcolor='lightgrey'),
        yaxis=dict(gridcolor='lightgrey', zeroline=True, zerolinecolor='black', zerolinewidth=1),
        plot_bgcolor='rgba(240, 240, 240, 0.95)'  # Light background
    )

    # Combine errors, calculating median, min, max, and midpoint
    combined_error = np.concatenate((over_error, under_error))

    median_combined_error = np.nanmedian(combined_error)  # Get median of combined array
    min_combined_error = np.nanmin(combined_error)  # Get minimum of combined array
    max_combined_error = np.nanmax(combined_error)  # Get maximum of combined array
    midpoint_combined_error = (min_combined_error + max_combined_error) / 2  # Get midpoint

    # Prepare tick labels for predicted and actual values
    tick_labels = []
    tick_vals = []
    for actual, predicted, diff in zip(actual_values, predicted_values, differences):
        if not np.isnan(diff):
            tick_vals.append(diff)
            tick_labels.append(f'Pred: {predicted}, Actual: {actual}')

    # Set tick values and labels, only show every other tick for legibility
    fig.update_xaxes(tickvals=tick_vals[::2], ticktext=tick_labels[::2])  # Show every other tick for clarity

    # Add annotations for sections, positioned in the center of the rectangles
    fig.add_annotation(
        y=midpoint_combined_error, 
        x=-50,
        text="Under-Prediction",
        showarrow=False,
        font=dict(size=14, color="black"),
        align="center"
    )
    fig.add_annotation(
        y=midpoint_combined_error, 
        x=50,
        text="Over-Prediction",
        showarrow=False,
        font=dict(size=14, color="black"),
        align="center"
    )

    # Add a shaded area for under-prediction
    fig.add_shape(
        type="rect",
        x0=-np.max(np.abs(differences)), y0=0, x1=0, y1=midpoint_combined_error * 2,
        fillcolor="rgba(0, 128, 255, 0.2)", line=dict(color="rgba(0, 128, 255, 1)", width=1),
        layer="below",  # Ensure it appears below the scatter plots
    )

    # Add a shaded area for over-prediction
    fig.add_shape(
        type="rect",
        x0=0, y0=-np.max(over_error), x1=np.max(np.abs(differences)), y1=midpoint_combined_error * 2,
        fillcolor="rgba(255, 165, 0, 0.2)", line=dict(color="rgba(255, 165, 0, 1)", width=1),
        layer="below",  # Ensure it appears below the scatter plots
    )

    # Create HTML for embedding
    graph_id = str(uuid.uuid4())
    html_str = f"""
    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {{MathJaxConfig: 'local'}};</script>
        <div id="{graph_id}" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {{}};
            if (document.getElementById("{graph_id}")) {{
                Plotly.newPlot(
                    "{graph_id}",
                    {fig.to_json()}
                )
            }};
        </script>
    </div>
    """
    
    # Display the figure in the notebook or Python environment
    fig.show()

    # Return the HTML string for embedding
    return html_str


In [3]:
def mean_absolute_error(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    return mae  # Return the mean absolute error

html_str = plot_error_symmetry(mean_absolute_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="3e8e2c7c-dbf5-48ab-9fc8-3e5ad2a823c4" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("3e8e2c7c-dbf5-48ab-9fc8-3e5ad2a823c4")) {
                Plotly.newPlot(
                    "3e8e2c7c-dbf5-48ab-9fc8-3e5ad2a823c4",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: 10, Actual: 100, Error: nan","Pred: 20, Actual: 90, Error: nan","Pred: 30, Actual: 80, Error: nan","Pred: 40, Actual: 70, Error: nan","Pred: 50, Actual: 60, Error: nan","Pred: 60, Actual: 50, Error: 10","Pred: 70, Actual: 40, Error: 30","Pred: 80

In [4]:
def median_absolute_error(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    mae = np.median(np.abs(actual - pred))
    return mae  # Return the median absolute error

html_str = plot_error_symmetry(median_absolute_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="0d7d4a48-b28d-435c-82e3-261065dbb3e8" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("0d7d4a48-b28d-435c-82e3-261065dbb3e8")) {
                Plotly.newPlot(
                    "0d7d4a48-b28d-435c-82e3-261065dbb3e8",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: 10, Actual: 100, Error: nan","Pred: 20, Actual: 90, Error: nan","Pred: 30, Actual: 80, Error: nan","Pred: 40, Actual: 70, Error: nan","Pred: 50, Actual: 60, Error: nan","Pred: 60, Actual: 50, Error: 10","Pred: 70, Actual: 40, Error: 30","Pred: 80

In [5]:
def geometric_mean_absolute_error(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    absolute_errors = np.abs(actual - pred)
    gmae = np.prod(absolute_errors) ** (1 / len(absolute_errors))
    return gmae  # Return the geometric mean absolute error

html_str = plot_error_symmetry(geometric_mean_absolute_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="4d04aafa-1d2c-4289-97aa-e28fc2c7f5b5" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("4d04aafa-1d2c-4289-97aa-e28fc2c7f5b5")) {
                Plotly.newPlot(
                    "4d04aafa-1d2c-4289-97aa-e28fc2c7f5b5",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: 10, Actual: 100, Error: nan","Pred: 20, Actual: 90, Error: nan","Pred: 30, Actual: 80, Error: nan","Pred: 40, Actual: 70, Error: nan","Pred: 50, Actual: 60, Error: nan","Pred: 60, Actual: 50, Error: 10","Pred: 70, Actual: 40, Error: 30","Pred: 80

In [6]:
def squared_error(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    mse = np.mean((actual - pred) ** 2)
    return mse  # Return the mean squared error

html_str = plot_error_symmetry(squared_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="dcb8a5df-bfab-4252-9a28-97bc237afe42" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("dcb8a5df-bfab-4252-9a28-97bc237afe42")) {
                Plotly.newPlot(
                    "dcb8a5df-bfab-4252-9a28-97bc237afe42",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: 10, Actual: 100, Error: nan","Pred: 20, Actual: 90, Error: nan","Pred: 30, Actual: 80, Error: nan","Pred: 40, Actual: 70, Error: nan","Pred: 50, Actual: 60, Error: nan","Pred: 60, Actual: 50, Error: 100","Pred: 70, Actual: 40, Error: 900","Pred: 

In [7]:
def percentage_error(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    # Avoid division by zero by replacing zeros in actual with a small value
    actual = np.where(actual == 0, 1e-10, actual)
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    return mape  # Return the mean absolute percentage error

html_str = plot_error_symmetry(percentage_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="d865a070-bb5d-4ab3-90f8-bfbfb8bb378d" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("d865a070-bb5d-4ab3-90f8-bfbfb8bb378d")) {
                Plotly.newPlot(
                    "d865a070-bb5d-4ab3-90f8-bfbfb8bb378d",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: 10, Actual: 100, Error: nan","Pred: 20, Actual: 90, Error: nan","Pred: 30, Actual: 80, Error: nan","Pred: 40, Actual: 70, Error: nan","Pred: 50, Actual: 60, Error: nan","Pred: 60, Actual: 50, Error: 20","Pred: 70, Actual: 40, Error: 75","Pred: 80

In [8]:

def symmetric_percentage_error(actual, forecast):
    actual, forecast = np.array(actual), np.array(forecast)
    numerator = np.abs(forecast - actual)
    denominator = (np.abs(actual) + np.abs(forecast)) / 2
    smape = (np.mean(numerator / denominator)) * 100
    return smape  # Return the symmetric mean absolute percentage error

html_str = plot_error_symmetry(symmetric_percentage_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="f05225fc-8bdf-4121-b42e-1c5000c39630" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("f05225fc-8bdf-4121-b42e-1c5000c39630")) {
                Plotly.newPlot(
                    "f05225fc-8bdf-4121-b42e-1c5000c39630",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: 10, Actual: 100, Error: nan","Pred: 20, Actual: 90, Error: nan","Pred: 30, Actual: 80, Error: nan","Pred: 40, Actual: 70, Error: nan","Pred: 50, Actual: 60, Error: nan","Pred: 60, Actual: 50, Error: 18","Pred: 70, Actual: 40, Error: 55","Pred: 80

In [29]:
def plot_error_single_value_symmetry(error_func):
    """
    Parameters:
    - error_func: function to calculate error given true and predicted values (e.g., mse)
    """
    graph_id = str(uuid.uuid4())
    # Set a fixed actual value
    actual_value = 100
    # Create a range of predicted values from 100 to -100 in decrements of 10
    predicted_values = np.arange(-100, 500, 10)
    # Create an array of the same actual value to match the shape of predicted values
    actual_values = np.full_like(predicted_values, actual_value)

    # Calculate differences centered around 0
    differences = predicted_values - actual_values
    over_diff = np.where(differences > 0, differences, np.nan)  # Positive differences (over-prediction)
    under_diff = np.where(differences < 0, differences, np.nan)  # Negative differences (under-prediction)

    # Calculate errors for each case
    over_error = [
        int(np.round(error_func(np.array([actual]), np.array([pred])))) if not np.isnan(d) else np.nan 
        for actual, pred, d in zip(actual_values, predicted_values, over_diff)
    ]
    under_error = [
        int(np.round(error_func(np.array([actual]), np.array([pred])))) if not np.isnan(d) else np.nan 
        for actual, pred, d in zip(actual_values, predicted_values, under_diff)
    ]

    # Create a Plotly figure
    fig = go.Figure()

    # Add trace for over-prediction errors
    fig.add_trace(go.Scatter(
        x=over_diff,
        y=over_error,
        mode='lines+markers',
        marker=dict(size=10, color='rgba(255, 165, 0, 0.8)', line=dict(width=1, color='rgba(255, 165, 0, 1)')),
        name='Over-Prediction Error',
        text=[f'Pred: {pred}, Actual: {actual_value}, Error: {error}' 
              for pred, error in zip(predicted_values, over_error)],
        hoverinfo='text'
    ))

    # Add trace for under-prediction errors
    fig.add_trace(go.Scatter(
        x=under_diff,
        y=under_error,
        mode='lines+markers',
        marker=dict(size=10, color='rgba(0, 128, 255, 0.8)', line=dict(width=1, color='rgba(0, 128, 255, 1)')),
        name='Under-Prediction Error',
        text=[f'Pred: {pred}, Actual: {actual_value}, Error: {error}' 
              for pred, error in zip(predicted_values, under_error)],
        hoverinfo='text'
    ))

    # Update layout for better appearance
    fig.update_layout(
        title=f"Impact on Over- and Under-Prediction for a Fixed Actual Value of 100",
        title_x=0.5,
        title_font=dict(size=16, family='"Segoe UI", "Roboto", "Oxygen", "Ubuntu", "Droid Sans", "Helvetica Neue", sans-serif', weight='bold'),
        xaxis_title="Difference between Predicted and Actual Values",
        yaxis_title="Error",
        template="plotly_white",
        font=dict(size=14, family='"Segoe UI", "Roboto", "Oxygen", "Ubuntu", "Droid Sans", "Helvetica Neue", sans-serif'),
        showlegend=False,
        xaxis=dict(tickmode='array', gridcolor='lightgrey'),
        yaxis=dict(gridcolor='lightgrey', zeroline=True, zerolinecolor='black', zerolinewidth=1),
        plot_bgcolor='rgba(240, 240, 240, 0.95)'
    )

    # Display the figure
    fig.show()

    html_str = f"""
    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {{MathJaxConfig: 'local'}};</script>
        <div id="{graph_id}" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {{}};
            if (document.getElementById("{graph_id}")) {{
                Plotly.newPlot(
                    "{graph_id}",
                    {fig.to_json()}
                )
            }};
        </script>
    </div>
    """
    return html_str

In [30]:
html_str = plot_error_single_value_symmetry(symmetric_percentage_error)
print(html_str)


    <div style="width: 100%; height: auto;">
        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <div id="9230fecd-e44a-498c-b5e1-aa046c908356" class="plotly-graph-div" style="width: 100%; height: auto;"></div>
        <script type="text/javascript">
            window.PLOTLYENV=window.PLOTLYENV || {};
            if (document.getElementById("9230fecd-e44a-498c-b5e1-aa046c908356")) {
                Plotly.newPlot(
                    "9230fecd-e44a-498c-b5e1-aa046c908356",
                    {"data":[{"hoverinfo":"text","marker":{"color":"rgba(255, 165, 0, 0.8)","line":{"color":"rgba(255, 165, 0, 1)","width":1},"size":10},"mode":"lines+markers","name":"Over-Prediction Error","text":["Pred: -100, Actual: 100, Error: nan","Pred: -90, Actual: 100, Error: nan","Pred: -80, Actual: 100, Error: nan","Pred: -70, Actual: 100, Error: nan","Pred: -60, Actual: 100, Error: nan","Pred: -50, Actual: 100, Error: nan","Pred: -40, Actual: 100, Error